In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/yolov7oc2'

In [ ]:
import cv2

In [ ]:
def plotbbox(imgpath,labpath):
  import matplotlib.pyplot as plt
  from google.colab.patches import cv2_imshow
  img1=plt.imread(imgpath)
  f1=open(labpath,'r')
  lab1=f1.readlines()
  shape1=img1.shape
  print(shape1)
  str1=lab1[0].split()
  print(lab1)
  x1=shape1[1]*float(str1[1])
  x2=shape1[0]*float(str1[2])
  x3=shape1[1]*float(str1[3])
  x4=shape1[0]*float(str1[4])
  color = (255, 0, 0)
  
  # Line thickness of 2 pxint(x1+(x3/2))  int(x2-(x4/2))
  thickness = 2
  ib1=cv2.rectangle(img1,(int(x1-(x3/2)),int(x2-(x4/2))),(int(x1+(x3/2)),int(x2+(x4/2))) , color, thickness)
  # ib1=cv2.rectangle(img1, (0,0), (5,5), color, thickness)
  cv2_imshow(ib1)
  return ib1
  

In [ ]:
class ActivationsAndGradients:
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform
        self.handles = []
        for target_layer in target_layers:
            self.handles.append(
                target_layer.register_forward_hook(self.save_activation))
            # Because of https://github.com/pytorch/pytorch/issues/61519,
            # we don't use backward hook to record gradients.
            self.handles.append(
                target_layer.register_forward_hook(self.save_gradient))

    def save_activation(self, module, input, output):
        activation = output

        if self.reshape_transform is not None:
            activation = self.reshape_transform(activation)
        self.activations.append(activation.cpu().detach())

    def save_gradient(self, module, input, output):
        if not hasattr(output, "requires_grad") or not output.requires_grad:
            # You can only register hooks on tensor requires grad.
            return

        # Gradients are computed in reverse order
        def _store_grad(grad):
            if self.reshape_transform is not None:
                grad = self.reshape_transform(grad)
            self.gradients = [grad.cpu().detach()] + self.gradients

        output.register_hook(_store_grad)

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        return self.model(x)

    def release(self):
        for handle in self.handles:
            handle.remove()



In [ ]:
def getpaths():
  imgpaths=[]
  labpaths=[]
  path1='custom_dataset/images/val'
  path2='custom_dataset/labels/val'
  for x in os.listdir(path1):
    imgpaths.append(path1+'/'+x)
  for x in os.listdir(path2):
    labpaths.append(path2+'/'+x)
  return imgpaths,labpaths

In [ ]:
imgwithbox1=plotbbox('/content/drive/MyDrive/yolov7oc2/custom_dataset/images/val/sanjaykumarimage 0.jpeg','/content/drive/MyDrive/yolov7oc2/custom_dataset/labels/val/sanjaykumarimage 0.txt')

In [ ]:
from models.experimental import attempt_load
from utils.torch_utils import TracedModel

wpath='/content/drive/MyDrive/yolov7oc2/runs/train/yolov7-custom/weights/best.pt'

import torch

device=torch.device('cpu')

model = attempt_load(wpath, map_location=device)


In [ ]:
layers=[]
n1=0
for x in range(len(model.model)):
  layers.append(model.model[x])

In [ ]:
y1=ActivationsAndGradients(model,target_layers=layers[0:104],reshape_transform=None)

In [ ]:
import numpy as np
import cv2

def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)





In [ ]:
gresults=['sashirajimage 1.jpeg','soorsinghimage 2.jpeg','surajmalikimage 1.jpeg','ramalingamimage 2.jpeg']

In [ ]:

for picr1 in gresults:
  import cv2
  im0 = cv2.imread('/content/drive/MyDrive/yolov7oc2/custom_dataset/images/val/'+picr1)

  img=letterbox(im0,640,32)[0]

  img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
  img = np.ascontiguousarray(img)

  import torch
  device=torch.device('cpu')
  img = torch.from_numpy(img).to(device)
  img = img.float()  # uint8 to fp16/32
  img /= 255.0  # 0 - 255 to 0.0 - 1.0
  if img.ndimension() == 3:
      img = img.unsqueeze(0)

  img.requires_grad=True

  outputs1=y1(img)

  # from utils.general import apply_classifier
  # from utils.general import non_max_suppression
  # pred = non_max_suppression(outputs1[0],0.25,0.45)
        
  # pred = apply_classifier(pred, modelc, img, im0)

  outputs1[0][0][:,4].max().backward()


  backprop1=torch.clone(img.grad.detach())
  backprop1=torch.relu(backprop1)
  backprop1=torch.squeeze(backprop1)


  r1=torch.rand((backprop1.shape[1],backprop1.shape[2],3))
  r1[:,:,0]=backprop1[0,:,:]
  r1[:,:,1]=backprop1[1,:,:]
  r1[:,:,2]=backprop1[2,:,:]


  backprop1=r1
  backprop1=(backprop1-backprop1.min())/backprop1.max()

  from google.colab.patches import cv2_imshow
  cv2_imshow(backprop1.numpy()*255)


  # plt.imsave('/content/drive/MyDrive/yolov7oc2/finoutputs2/guidbackprop'+picr1,(backprop1.numpy()*255).astype(np.uint8))
  backpropimg=(backprop1.numpy()*255).astype(np.uint8).copy()

  weights1=torch.mean(y1.gradients[101],axis=(2,3))
  gradcam=torch.clone(y1.activations[101].detach())
  for x in range(gradcam.shape[1]):
    gradcam[0,x,:,:]=weights1[0][x].detach()*gradcam[0,x,:,:]

  gradcamf=torch.sum(gradcam[0].detach(),axis=0)
  gradcamf=torch.relu(gradcamf)

  import torchvision
  transform1=torchvision.transforms.functional.resize(gradcamf.view([1,gradcamf.shape[0],gradcamf.shape[1]]),(img.shape[2],img.shape[3]))

  gradcam=transform1.view([img.shape[2],img.shape[3]])

  gradcam=(gradcam-gradcam.min())/gradcam.max()

  cv2_imshow(gradcam.detach().numpy()*255)

  # plt.imsave('/content/drive/MyDrive/yolov7oc2/finoutputs2/gradcammap'+picr1,gradcam.detach().numpy()*255)

  gradcamimg=(gradcam.detach().numpy()*255).copy()

  for x in range(3):
    backprop1[:,:,0]=gradcam*backprop1[:,:,0]
    backprop1[:,:,1]=gradcam*backprop1[:,:,1]
    backprop1[:,:,2]=gradcam*backprop1[:,:,2]

  # plt.imsave('/content/drive/MyDrive/yolov7oc2/finoutputs2/guidedgradcam'+picr1,(backprop1.detach().numpy()*255*255).astype(np.uint8))
  guidedgradcam=(backprop1.detach().numpy()*255*255).astype(np.uint8).copy()

  imgforgrad=torch.clone(img.detach()).squeeze()
  for x in range(3):
    imgforgrad[x,:,:]=imgforgrad[x,:,:]*gradcam*255*255

  r2=torch.rand((img.shape[2],img.shape[3],3))
  r2[:,:,0]=imgforgrad[0,:,:]
  r2[:,:,1]=imgforgrad[1,:,:]
  r2[:,:,2]=imgforgrad[2,:,:]


  # plt.imsave('/content/drive/MyDrive/yolov7oc2/finoutputs2/gradcamimp'+picr1,r2.detach().numpy()/(255*255))
  gradcamsuperimpose=(r2.detach().numpy()/(255*255)).copy()

  rx=torch.rand((img.shape[2],img.shape[3],3))
  rx[:,:,0]=img[0][0]
  rx[:,:,1]=img[0][1]
  rx[:,:,2]=img[0][2]
  # plt.imsave('/content/drive/MyDrive/yolov7oc2/finoutputs2/'+picr1,(rx.detach().numpy()))
  originimg=rx.detach().numpy().copy()
  dirx='/content/drive/MyDrive/yolov7oc2/runs/detect/exp7'
  detectimg=cv2.imread(dirx+'/'+picr1)
  

  fig = plt.figure(figsize=(10, 10))
    
  # setting values to rows and column variables
  rows = 2
  columns = 3
    
  # reading images
  Image1 = originimg
  Image2 = detectimg
  Image3 = backpropimg
  Image4 = gradcamimg
  Image5 = gradcamsuperimpose
  Image6 = guidedgradcam
    
  # Adds a subplot at the 1st position
  fig.add_subplot(rows, columns, 1)
    
  # showing image
  plt.imshow(Image1)
  plt.axis('off')
  plt.title("origin")
    
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
    
  # showing image
  plt.imshow(Image2)
  plt.axis('off')
  plt.title("detectpic")
    
  # Adds a subplot at the 3rd position
  fig.add_subplot(rows, columns, 3)
    
  # showing image
  plt.imshow(Image3)
  plt.axis('off')
  plt.title("backproppic")
    
  # Adds a subplot at the 4th position
  fig.add_subplot(rows, columns, 4)
    
  # showing image
  plt.imshow(Image4)
  plt.axis('off')
  plt.title("gradcampic")

  fig.add_subplot(rows, columns, 5)
    
  # showing image
  plt.imshow(Image5)
  plt.axis('off')
  plt.title("gradcamsuperimpose")

  fig.add_subplot(rows, columns, 6)
    
  # showing image
  plt.imshow(Image6)
  plt.axis('off')
  plt.title("guidedgradcam")

  plt.savefig('/content/drive/MyDrive/yolov7oc2/finoutputs3/'+picr1)